In [8]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import pickle

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'
playlists = sp.user_playlists(user_id)
playlists = playlists['items']





playlist_id = '3YwPUeHZqBrFzLzP5BF8ND'





results = sp.playlist_tracks(playlist_id)
tracks = results['items']
track_ids = []
for track in tracks:
    track_ids.append(track['track']['id'])
track_ids

columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['danceability',
'energy',
'key',
'loudness',
'mode',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo',
'id',
'duration_ms',
'time_signature',
'track_title',
'album_title',
'album_artist',
'track_number',
'total_tracks']

playlist_data = pd.DataFrame(columns=headings)

playlist_length = len(track_ids)

i = 0
for track in track_ids:
    results = sp.audio_features(track)
    if results[0]: 
        features = results[0]
    
    features_matrix = pd.DataFrame.from_records(features, index=[0])
    
    # Remove unneeded columns
    features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
    # print(features_matrix)

    # Add track title
    track_title = tracks[i]['track']['name']
    features_matrix['track_title'] = track_title

    # Add album title
    album_title = tracks[i]['track']['album']['name']
    features_matrix['album_title'] = album_title

    # Add artist name
    name = tracks[i]['track']['album']['artists'][0]['name']
    features_matrix['album_artist'] = name

    # Add track number and total tracks
    features_matrix['track_number'] = tracks[i]['track']['track_number']
    features_matrix['total_tracks'] = playlist_length

    # Add album as a row to the data df
    # print(features_matrix,'\n')
    # print(data)
    
    playlist_data = pd.concat([playlist_data, features_matrix])
    i += 1

playlist_data = playlist_data.reset_index(drop=True)
# playlist_data.to_csv('./data/playlist_data.csv')
playlist_data_full = pd.DataFrame(playlist_data)



# Remove unnecessary testing columns
columns_to_remove = ['id',
                    'track_title',
                    'album_title',
                    'album_artist',
                    'track_number',
                    'total_tracks',
                    'key',
                    'mode',
                    'duration_ms',
                    'time_signature']
playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)



# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

## Load trained model
model = pickle.load(open('./regression/bayesian.sav', 'rb'))
playlist_data_full['order'] = model.predict(playlist_data.values)
playlist_data_full = playlist_data_full.sort_values(by=['order'])
# playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
playlist_data_full

# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST

C:\Users\crisp\anaconda3\envs\AI_general\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator BayesianRidge from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'snapshot_id': 'NjMsMzEwOTc5NTE1OTE5MDlmMjUxOGI2MzdlYjViM2M4MTY2ZWZjZmJhYg=='}

In [53]:
## Load trained model
model = pickle.load(open('./regression/en.sav', 'rb'))
playlist_data_full['order'] = model.predict(playlist_data.values)
playlist_data_full = playlist_data_full.sort_values(by=['order'])
# playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
playlist_data_full

C:\Users\crisp\anaconda3\envs\AI_general\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator ElasticNet from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,order
6,0.554,0.864,2,-4.403,1,0.0889,0.00166,0,0.119,0.305,126.975,3rnBqIIoxOpQ0p9BeW3NT4,201260,4,Back To Life,Back To Life,DubVision,0.000000,10,0.483739
7,0.62,0.627,9,-5.889,1,0.0259,0.0317,0,0.172,0.164,90.955,26AuyrZGzWWiYZPSd3XBIg,224280,4,Bloodstream,Memories...Do Not Open,The Chainsmokers,0.285714,10,0.484612
1,0.28,0.449,11,-7.736,1,0.0292,0.638,0.701,0.105,0.103,136.4,2DI0fZ4QZmLtapszYaoG6F,236093,4,Atlas - From “The Hunger Games: Catching Fire”...,Atlas (From “The Hunger Games: Catching Fire” ...,Coldplay,0.000000,10,0.485528
5,0.632,0.805,0,-5.188,1,0.045,0.0646,0.000058,0.161,0.429,93.009,6JsfvqClYnc2R5yZ67C2ch,217793,4,"Cross My Mind, Pt. 2 (feat. Kiiara)",Cross My Mind: The Mixtape,A R I Z O N A,0.000000,10,0.487532
9,0.895,0.453,0,-13.987,1,0.0756,0.00196,0.374,0.0982,0.257,122.151,0KQh7AuuZvpTKWhcJa8Pbr,239227,4,Funkytown,Casablanca Records Greatest Hits,Various Artists,0.571429,10,0.489469
3,0.671,0.874,0,-4.188,1,0.0448,0.0829,0.000027,0.294,0.747,109.96,22jEGuVPtvtMqKPuXdOVCh,152203,4,SAD (Clap Your Hands),SAD / Scatterbrain,Young Rising Sons,0.000000,10,0.498457
4,0.753,0.782,7,-5.826,1,0.118,0.0106,0.000004,0.111,0.15,100.006,77Dn6Y5SzjCzfXLjy89dYB,184800,4,Broken Child (feat. JDAM & LissA),Broken Child (feat. JDAM & LissA),Not Your Dope,0.000000,10,0.502658
0,0.859,0.625,1,-5.437,1,0.0914,0.00538,0.000124,0.184,0.636,149.948,5y2K7jr9M8t1HRAu9jzssT,219737,4,No Security,No Security,Skepta,0.000000,10,0.507392
2,0.643,0.445,9,-15.23,0,0.043,0.682,0.931,0.087,0.0714,105.0,1LYipMFKz8X6fNU7RJvvVW,165653,4,Last Kiss,Glorious,ARTY,1.000000,10,0.519468
8,0.341,0.293,9,-12.408,1,0.0356,0.756,0.000023,0.0833,0.178,93.294,3js9UOPVsguIDI2WyycfRJ,249213,4,Doing Alright - Remastered 2011,Queen,Queen,0.142857,10,0.519596


In [34]:
from sklearn.preprocessing import MinMaxScaler
segmentColumnsToNormalise = ['track_number','order']
playlist_data_full[segmentColumnsToNormalise] = MinMaxScaler().fit_transform(playlist_data_full[segmentColumnsToNormalise])
playlist_data_full

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,order
3,0.671,0.874,0,-4.188,1,0.0448,0.0829,0.000027,0.294,0.747,109.96,22jEGuVPtvtMqKPuXdOVCh,152203,4,SAD (Clap Your Hands),SAD / Scatterbrain,Young Rising Sons,0.000000,10,0.000000
5,0.632,0.805,0,-5.188,1,0.045,0.0646,0.000058,0.161,0.429,93.009,6JsfvqClYnc2R5yZ67C2ch,217793,4,"Cross My Mind, Pt. 2 (feat. Kiiara)",Cross My Mind: The Mixtape,A R I Z O N A,0.000000,10,0.024345
2,0.643,0.445,9,-15.23,0,0.043,0.682,0.931,0.087,0.0714,105.0,1LYipMFKz8X6fNU7RJvvVW,165653,4,Last Kiss,Glorious,ARTY,1.000000,10,0.049869
9,0.895,0.453,0,-13.987,1,0.0756,0.00196,0.374,0.0982,0.257,122.151,0KQh7AuuZvpTKWhcJa8Pbr,239227,4,Funkytown,Casablanca Records Greatest Hits,Various Artists,0.571429,10,0.105769
4,0.753,0.782,7,-5.826,1,0.118,0.0106,0.000004,0.111,0.15,100.006,77Dn6Y5SzjCzfXLjy89dYB,184800,4,Broken Child (feat. JDAM & LissA),Broken Child (feat. JDAM & LissA),Not Your Dope,0.000000,10,0.159805
6,0.554,0.864,2,-4.403,1,0.0889,0.00166,0,0.119,0.305,126.975,3rnBqIIoxOpQ0p9BeW3NT4,201260,4,Back To Life,Back To Life,DubVision,0.000000,10,0.410446
7,0.62,0.627,9,-5.889,1,0.0259,0.0317,0,0.172,0.164,90.955,26AuyrZGzWWiYZPSd3XBIg,224280,4,Bloodstream,Memories...Do Not Open,The Chainsmokers,0.285714,10,0.527608
1,0.28,0.449,11,-7.736,1,0.0292,0.638,0.701,0.105,0.103,136.4,2DI0fZ4QZmLtapszYaoG6F,236093,4,Atlas - From “The Hunger Games: Catching Fire”...,Atlas (From “The Hunger Games: Catching Fire” ...,Coldplay,0.000000,10,0.659642
0,0.859,0.625,1,-5.437,1,0.0914,0.00538,0.000124,0.184,0.636,149.948,5y2K7jr9M8t1HRAu9jzssT,219737,4,No Security,No Security,Skepta,0.000000,10,0.996429
8,0.341,0.293,9,-12.408,1,0.0356,0.756,0.000023,0.0833,0.178,93.294,3js9UOPVsguIDI2WyycfRJ,249213,4,Doing Alright - Remastered 2011,Queen,Queen,0.142857,10,1.000000


In [24]:
from sklearn.metrics import mean_squared_error
y_true = playlist_data_full['track_number']
y_pred = playlist_data_full['order']
mean_squared_error(y_true, y_pred)

0.20814743554271892

Rearrange

In [25]:
# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))


# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST

{'snapshot_id': 'NjEsOTNlNmM4N2I3OTE3MThmYTM1YWMzOTE5NjJmYzhhN2FkZDI0ZWJhNQ=='}